<a href="https://colab.research.google.com/github/NNLogic/NNPosterPublic/blob/main/%5BNNPoster%5DJson_Images_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#説明 Description
PDF/PNG/JPGからNNPoster用のJSON画像ファイルを作成するスクリプトになります。

Betaテスト版です。

★[1]→[2]→[3]→[4]の順番に実行してください。


※NNPosterに同梱されているUnityのスクリプトでは、crunch圧縮によりファイルサイズを小さくできます。

※スライドを変換する場合、max_widthは1024を推奨しています。


The script to create a JSON image file for NNPoster from PDF/PNG/JPG:

"This is a beta test version.

★Please execute the steps in the following order: [1] → [2] → [3] → [4].

※The Unity script included in NNPoster allows for file size reduction through crunch compression.

※When converting slides, we recommend using a max_width of 1024."

In [ ]:
# @title #[1] 変換設定 / Settings
rename = "on" #@param ["on", "off"]
#@markdown ※ファイル名を表示する場合、末尾の数字をサムネイル位置にする場合、rename=offにしてください。
#@markdown
#@markdown If you want to display the file name with the thumbnail position at the end, please set "rename=off."

max_width = '1024' #@param [1024, 2048, 4096, 8192] {allow-input: true}

#[2] ファイルのロード Loading Files

[2]は次の読込方法により一つだけ実行してください。

・Googleドライブからファイルをコピーする場合

　→　[2-A]を実行後、PNG/JPGは[2-A1]、pdfは[2-A2]を実行してください。

・ローカルコンピューターからPNG/JPGをアップロードする場合

　→　[2-B1]を実行してください。

・ローカルコンピューターからPDFをアップロードする場合

　→　[2-B2]を実行してください。

Choose only one method from the following for [2] based on the loading method.

・If you are copying files from Google Drive:

　→　Execute [2-A] first, then [2-A1] for PNG/JPG or [2-A2] for PDF.

・If you are uploading PNG and JPG from your local computer:

　→　Execute [2-B1].

・If you are uploading a PDF from your local computer:

　→　Execute [2-B2].

In [ ]:
#@title [2-A] Googleドライブをマウントする　Mounting Google Drive
%cd  /content

import ipywidgets as widgets
from google.colab import drive

if 'drive' in locals():
  drive.flush_and_unmount()

drive.mount('/content/drive')

In [ ]:
#@title [2-A1] Googleドライブのフォルダからpng, jpgをコピーする　Copying PNG and JPG files from Google Drive folder
%cd  /content

if 'rename' not in locals():
    rename = "on"

from pkgutil import extend_path
import os
import shutil

!rm -f *.pdf *.png *.json

folder_path = "/content/drive/MyDrive" #@param {type:"string"}

copy_dest = "/content"  # コピー先のディレクトリパスを指定してください
os.chdir(folder_path)
image_files = [file for file in os.listdir() if file.endswith('.png') or file.endswith('.jpg')]

no = 0
for file in image_files:
    no = no + 1
    src_path = os.path.join(folder_path, file)
    dest_path = os.path.join(copy_dest, file)
    shutil.copy(src_path, dest_path)
    if rename == "on":
      file_name, file_extension = os.path.splitext(dest_path)
      new_name = "image-"+str(no)+file_extension
      os.rename(dest_path, os.path.join(os.path.dirname(dest_path), new_name))

In [ ]:
#@title [2-A2] GoogleドライブからPDFをコピーする　Copying PDF from Google Drive
%cd  /content

if 'rename' not in locals():
    rename = "on"

if 'max_width' not in locals():
    max_width = 1024

import os
import shutil

!rm -f *.pdf *.png *.json

pdf_path = "/content/drive/MyDrive" #@param {type:"string"}
copy_dest = "/content"  # コピー先のディレクトリパスを指定してください

file_name = os.path.basename(pdf_path)
base_file_name = os.path.splitext(file_name)[0]

dest_path = os.path.join(copy_dest, file_name)
shutil.copy(pdf_path, dest_path)

!apt-get install -y poppler-utils

if rename == "on":
  !pdftoppm -scale-to $max_width -png "$file_name" image
else
  !pdftoppm -scale-to $max_width -png "$file_name" "$base_file_name"


In [ ]:
#@title [2-B1]  png/jpgをアップロードする　Uploading png/jpg
!cd /content
%cd  /content

if 'rename' not in locals():
    rename = "on"

!rm -f *.pdf *.png *.json

from google.colab import files
import os

image_files = files.upload()
sorted_files = sorted(image_files)

no = 0
for file in sorted_files:
   if file.endswith(".png") or file.endswith(".jpg"):
      no = no + 1
      if rename == "on":
        file_name, file_extension = os.path.splitext(file)
        new_name = "image-"+str(no)+file_extension
        os.rename(file, os.path.join(os.path.dirname(file), new_name))

In [ ]:
#@title [2-B2] PDFをアップロードする　Uploading a PDF

!cd /content
%cd  /content

if 'rename' not in locals():
    rename = "on"

if 'max_width' not in locals():
    max_width = 1024

!rm -f *.pdf *.png *.json

from google.colab import files
import os

result = files.upload()
pdf_file_name = list(result.keys())[0]
base_file_name = os.path.splitext(pdf_file_name)[0]

!apt-get install -y poppler-utils

if rename == "on":
  !pdftoppm -scale-to $max_width -png "$pdf_file_name" image
else
  !pdftoppm -scale-to $max_width -png "$pdf_file_name" "$base_file_name"

#[3]JSONファイルへの変換 Converting to JSON file

In [ ]:
#@title [3]JSONファイルへの変換 Converting to JSON file
%cd  /content

!sudo apt-get install imagemagick
!pip install wand

import os, sys
from wand.image import Image
import datetime
import base64
import json
import re

if 'max_width' not in locals():
    max_width = 1024

if 'rename' not in locals():
    rename = "on"

no = 0
images = []
files = os.listdir("./")
sorted_files = sorted(files)
for file_name in sorted_files:
    if file_name.endswith(".png") or file_name.endswith(".jpg"):
      im = Image(filename=file_name)

      if im.width > int(max_width):
        target_width = int(max_width)
        target_height = int(im.height * (target_width / im.width))
        im.resize(target_width, target_height)

      im.flip()
      im.convert("RGBA")
      im.options['dds:mipmaps'] = "0"
      im.compression = "dxt5"
      im.format = 'dds'
      b64_txt = base64.b64encode(im.make_blob()[128:])

      type_str = "9" if im.compression == "dxt1" else "10" # type DXT1 9 DXT5 10

      match = re.search(r"\d+$", file_name)
      no = int(match.group()) if match else no + 1
      no = 200 if no > 200 else 1 if no < 1 else no

      image_dict = {"no":str(no),"name":os.path.splitext(file_name)[0],"type":type_str,"width":str(im.width),"height":str( im.height),"base64":b64_txt.decode('utf-8')};
      images.append(image_dict)

dict= {"version":"001","specified_index":"true" if rename == "off" else "false","images":images}

# output
current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y%m%d_%H%M%S")
json_file_name = formatted_datetime + '.json'
with open(json_file_name, "w") as outputFile:
  json.dump(dict, outputFile, indent=2 )

#[4]ファイルのダウンロード Downloading the file

In [ ]:
#@title [4]ファイルのダウンロード Downloading the file
from google.colab import files

files.download(json_file_name)